## URL to Comma Separated Value

In [1]:
import urllib.parse
import re
from pathlib import Path


class Line(str):
    def is_url(self) -> bool:
        return len(urllib.parse.urlparse(self).scheme) > 0

    def decode_url(self) -> "Line":
        return Line(urllib.parse.unquote(self))

    def strip(self) -> "Line":
        return Line(re.sub(
            "&[a-z]+=.+",
            "",
            self.replace(
                "https://www.google.com/search?q=",
                "",
            ).replace(
                ",",
                "",
            )).replace(
                "+",
                ",",
            )
        )


def decode_and_to_csv(path: Path) -> None:
    if path.is_file() and path.suffix == ".csv":
        with path.open(mode="r", encoding="utf_8_sig") as fp:
            lines: list[Line] = [Line(line) for line in fp.readlines()]
        if all([line.is_url() for line in lines]):
            print(f'"{path.as_posix()}"')
            with path.open(mode="w", encoding="utf_8_sig") as fp:
                fp.writelines([line.decode_url().strip() for line in lines])
    elif path.is_dir():
        for child in path.glob("*"):
            decode_and_to_csv(child)

decode_and_to_csv(Path("."))

"2. 荷重および応力の組合せ/2-1. 建築用材料の単位容積質量.csv"
"6. 高力ボルト接合部/6-1. ボルトの許容耐力（鋼構造設計規準）.csv"
"6. 高力ボルト接合部/6-2. 高力ボルトおよびリベットのピッチ、ゲージ、その他の標準.csv"
"7. 基礎構造/7-1. 地盤調査によらない場合の許容応力度（令93条）.csv"
"7. 基礎構造/7-5. 鋼管ぐいの断面性能表（一例）.csv"
"8. 各部構造/8-2. フラットデッキ（床型枠用鋼製デッキプレート）.csv"
"8. 各部構造/8-3. 合成スラブ構造用デッキプレート.csv"
"8. 各部構造/8-5. 母屋・根太.csv"


## Transpose Table

In [ ]:
import numpy as np
import io
import csv

def read_as_table(self: str) -> np.ndarray:
    with io.StringIO() as fp:
        fp.write(self)
        fp.seek(0)
        content: list[list[str]] = [row for row in csv.reader(fp)]
        num_col: int = max([len(row) for row in content])
        for row in content:
            while len(row) < num_col:
                row.append("")
        return np.array(content)

def to_text(self: np.ndarray) -> str:
    with io.StringIO(newline="") as fp:
        writer = csv.writer(fp)
        writer.writerows(self.tolist())
        fp.seek(0)
        return fp.read()

def transpose(filename: str) -> None:
    with open(filename, "r", encoding="utf_8_sig") as fp:
        tables: list[np.ndarray] = [read_as_table(table).T for table in fp.read().split("\n\n")]
    if all([min(table.shape) > 1 for table in tables]):
        print(filename)
        content: str = "\n\n".join([to_text(table) for table in tables])
        with open(filename, "w", encoding="utf_8_sig") as fp:
            fp.writelines(content)

for path in [
    # '8. 各部構造/8-2.2. JF75ME・JF75MW.csv',
    # '8. 各部構造/8-2.3. JF75・JF75W.csv',
    # '8. 各部構造/8-2.4. JF75α.csv',
    # '8. 各部構造/8-2.5. JF100M.csv',
    # '8. 各部構造/8-2.6. JF100.csv',
]:
    transpose(path)


8. 各部構造/8-2.2. JF75ME・JF75MW.csv
8. 各部構造/8-2.3. JF75・JF75W.csv
8. 各部構造/8-2.4. JF75α.csv
8. 各部構造/8-2.5. JF100M.csv
8. 各部構造/8-2.6. JF100.csv


In [1]:
import os
from pathlib import Path

def listdir(parent: str) -> list[str]:
    return [os.path.join(parent, child) for child in os.listdir(parent)]


def recurse_listdir(parent: Path) -> list[Path]:
    result: list[str] = []
    for child in parent.glob("*"):
        if child.is_file() and child.suffix == ".csv":
            result.append(child)
        elif child.is_dir():
            result += recurse_listdir(child)
    return result

sorted([file.as_posix() for file in recurse_listdir(Path("."))])


['2. 荷重および応力の組合せ/2-1. 建築用材料の単位容積質量.csv',
 '2. 荷重および応力の組合せ/2-2. 固定荷重（令84条）.csv',
 '2. 荷重および応力の組合せ/2-3. 積載荷重.csv',
 '5. 断面性能表/5-1.1. H形鋼.csv',
 '5. 断面性能表/5-1.2. H形鋼_部材種別.csv',
 '5. 断面性能表/5-10.1. UOE鋼管、板巻き鋼管.csv',
 '5. 断面性能表/5-10.2. 電縫鋼管.csv',
 '5. 断面性能表/5-10.3. シームレス鋼管.csv',
 '5. 断面性能表/5-11. 溶接四面ボックス.csv',
 '5. 断面性能表/5-12.1. 平鋼.csv',
 '5. 断面性能表/5-12.2. 鋼板の単位質量表.csv',
 '5. 断面性能表/5-3. CT形鋼.csv',
 '5. 断面性能表/5-4. 溶接軽量H形鋼.csv',
 '5. 断面性能表/5-5.1. 等辺山形鋼.csv',
 '5. 断面性能表/5-5.2. 不等辺山形鋼.csv',
 '5. 断面性能表/5-5.3. 不等辺不等厚山形鋼.csv',
 '5. 断面性能表/5-5.4. 溝形鋼.csv',
 '5. 断面性能表/5-5.5. I形鋼.csv',
 '5. 断面性能表/5-6. 角型鋼管（小径）.csv',
 '5. 断面性能表/5-7.1. STKR400, STKR490.csv',
 '5. 断面性能表/5-7.2. BCR295, JBCR385.csv',
 '5. 断面性能表/5-8. 冷間成形角形鋼管.csv',
 '5. 断面性能表/5-9. カクホット.csv',
 '6. 高力ボルト接合部/6-1.1. 高力ボルトの許容耐力等.csv',
 '6. 高力ボルト接合部/6-1.2. ボルトの許容耐力.csv',
 '6. 高力ボルト接合部/6-2.1. 形鋼のゲージ.csv',
 '6. 高力ボルト接合部/6-2.2. 高力ボルトの孔径・最小縁端距離・ピッチ.csv',
 '6. 高力ボルト接合部/6-2.3. 千鳥打ちのゲージとピッチ.csv',
 '6. 高力ボルト接合部/6-2.4. 形鋼に対する千鳥打ち.csv',
 '6. 高力ボルト接合部/6-6.1